# Utils

Utilities for visualisation and filtering.

In [ ]:
# | default_exp utils

In [ ]:
# | exporti

import jax
import jax.numpy as jnp
import numpy as np
from IPython.display import HTML, Audio
from matplotlib import pyplot as plt

In [ ]:
# | export


def to_db(x):
    return 20 * jnp.log10(x)


def safe_log(x, eps=1e-10):
    return jnp.log(x + eps)


def next_root(x):
    root = int(np.sqrt(x))
    if root * root == x:
        return root
    else:
        return root + 1

In [ ]:
# | export


def impulse_response(
    b: jnp.ndarray,  # numerators of the discrete transfer function
    a: jnp.ndarray,  # denominators of the discrete transfer function
    n=4410,
):
    """
    Compute the impulse response of a discrete time system

    Parameters
    ----------
    b : jnp.ndarray
        The numerator of the discrete transfer function, with shape (n_modes, n)
    a : jnp.ndarray
        The denominator of the discrete transfer function, with shape (n_modes, n)

    Returns
    -------
    jnp.ndarray
        The impulse response of the system, with shape (n_modes, n)

    """
    # sample the discrete time systems
    ret = jnp.fft.rfft(b, n=n) / jnp.fft.rfft(a, n=n)
    return jnp.fft.irfft(ret)

In [ ]:
# | export


def iir_filter_parallel(
    b,  # shape (n_modes, 3)
    a,  # shape (n_modes, 3)
    x,  # shape (n_samples,)
):
    """
    Efficient parallel IIR filter implementation using JAX.

    Parameters:
    -----------
    b : jnp.ndarray
        Numerator coefficients with shape (n_modes, 3)
    a : jnp.ndarray
        Denominator coefficients with shape (n_modes, 3)
    x : jnp.ndarray
        Input signal with shape (n_samples,)

    Returns:
    --------
    y : jnp.ndarray
        Output signal with shape (n_samples, n_modes)
    """
    n_modes = b.shape[0]

    # Extract coefficients
    b0 = b[:, 0]  # shape (n_modes,)
    b1 = b[:, 1]  # shape (n_modes,)
    b2 = b[:, 2]  # shape (n_modes,)
    a1 = a[:, 1]  # shape (n_modes,)
    a2 = a[:, 2]  # shape (n_modes,)

    # Initialize state for scan
    # State contains the last two outputs and inputs for each mode
    init_state = (
        jnp.zeros(shape=(n_modes,)),  # y[n-1]
        jnp.zeros(shape=(n_modes,)),  # y[n-2]
        0.0,  # x[n-1]
        0.0,  # x[n-2]
    )

    def scan_fn(state, x_n):
        y_n_1, y_n_2, x_n_1, x_n_2 = state

        # Compute new output
        y_n = b0 * x_n + b1 * x_n_1 + b2 * x_n_2 - a1 * y_n_1 - a2 * y_n_2

        # Update state for next iteration
        new_state = (y_n, y_n_1, x_n, x_n_1)

        return new_state, y_n

    # Run the scan
    _, y = jax.lax.scan(
        scan_fn,
        init_state,
        x,
    )

    return y

In [ ]:
# | export


def plot_poles_zeros(
    p: np.ndarray,  # Array of zeros or poles (any shape)
    ax=None,
    xlim=(-1.1, 1.1),  # Default x-axis limits
    ylim=(-1.1, 1.1),  # Default y-axis limits
    show_unit_circle=True,  # Option to show the unit circle
    save_path=None,  # Optional file path to save the plot
    **kwargs,
):
    """
    Plot poles and zeros on a 2D complex plane with customizable axis limits.

    Args:
        p (np.ndarray): Array of complex poles/zeros, will be flattened.
        ax (matplotlib.axes._subplots.AxesSubplot, optional): Pre-existing axis to plot on.
        xlim (tuple, optional): Limits for the x-axis (real part).
        ylim (tuple, optional): Limits for the y-axis (imaginary part).
        show_unit_circle (bool, optional): Whether to show the unit circle.
        save_path (str, optional): File path to save the plot.
        **kwargs: Additional keyword arguments passed to scatter plot.
    """
    # Flatten the array to ensure it is 1D
    p = p.flatten()

    # Create figure and axis if not provided
    fig, ax = plt.subplots(1, 1, figsize=(5, 5)) if ax is None else (None, ax)

    # Plot the unit circle if enabled
    if show_unit_circle:
        unit_circle = plt.Circle(
            (0.0, 0.0),
            1.0,
            color="gray",
            fill=False,
            label="Unit circle",
            linestyle="--",
        )
        ax.add_artist(unit_circle)

    # Plot the poles/zeros
    points = ax.scatter(p.real, p.imag, label="Eigenvalues", **kwargs)

    # Dashed gridlines
    gridlines = ax.get_xgridlines() + ax.get_ygridlines()
    for line in gridlines:
        line.set_linestyle("-.")

    ax.grid(True)

    # Axis labels
    ax.set_xlabel("Real part")
    ax.set_ylabel("Imaginary part")

    # Set axis limits
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    # Dashed lines for axes
    ax.plot([xlim[0], xlim[1]], [0, 0], linestyle="dashed", color="black", alpha=0.1)
    ax.plot([0, 0], [ylim[0], ylim[1]], linestyle="dashed", color="black", alpha=0.1)

    # Legend
    if show_unit_circle:
        ax.legend(
            [points, unit_circle],
            ["Eigenvalues", "Unit circle"],
            loc="best",
        )
    else:
        ax.legend(
            [points],
            ["Eigenvalues"],
            loc="best",
        )

    # Set equal aspect ratio
    # ax.set_aspect("equal")

    # Save plot if a path is provided
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches="tight")

    # Show the plot if created locally
    if fig:
        plt.show()

In [ ]:
# | export


def mel2hz(m):
    return 700 * (10 ** (m / 2595) - 1)


def hz2mel(h):
    return 2595 * np.log10(1 + h / 700)


def bark2hz(b):
    for ii in range(len(b)):
        if b[ii] < 2:
            b[ii] = (b[ii] - 0.3) / 0.85
        elif b[ii] > 20.1:
            b[ii] = (b[ii] + 0.22 * 20.1) / 1.22
    hz = 1960 * (b + 0.53) / (26.28 - b)
    return hz


def hz2bark(f):
    bark = 26.81 * f / (1960 + f) - 0.53
    for ii in range(len(bark)):
        if bark[ii] < 2:
            bark[ii] = 0.85 * bark[ii] + 0.3
        elif bark[ii] > 20.1:
            bark[ii] = 1.22 * bark[ii] - 4.422
    return bark

In [ ]:
# | export


def tf_freqs(
    b: jnp.ndarray,  # numerator   (n_modes, 3)
    a: jnp.ndarray,  # denominator (n_modes, 3)
    s: jnp.ndarray,  # frequency   (n_freqs,)
) -> jnp.ndarray:
    """
    Sample the frequency response of a transfer functions.

    Parameters
    ----------
    b : jnp.ndarray
        The numerator of the transfer function.
    a : jnp.ndarray
        The denominator of the transfer function.
    s : jnp.ndarray
        The frequencies to evaluate the transfer function at.

    Returns
    -------
    jnp.ndarray
        The frequency response of the transfer function.
    """

    num = b[:, 0][..., None] * s**2 + b[:, 1][..., None] * s + b[:, 2][..., None]
    den = a[:, 0][..., None] * s**2 + a[:, 1][..., None] * s + a[:, 2][..., None]
    return num / den

In [ ]:
# | export


def tf_freqz(
    b: jnp.ndarray,  # numerator   (n_modes, 3)
    a: jnp.ndarray,  # denominator (n_modes, 3)
    worN: jnp.ndarray,  # frequencies (n_freqs,)
    sample_rate: int,
):
    """
    Sample the frequency response of the transfer functions.

    Parameters
    ----------
    b : jnp.ndarray
        The coefficients of the numerator of the transfer function
    a : jnp.ndarray
        The coefficients of the denominator of the transfer function
    worN : jnp.ndarray
        The frequencies in Hz to sample the transfer function at
    sample_rate : int
        The sample rate in Hz of the transfer function

    Returns
    -------
    h : jnp.ndarray
        The frequency response of the transfer function
    """
    z = jnp.exp(2j * jnp.pi * worN / sample_rate)
    num = b[:, 0][..., None] * z**2 + b[:, 1][..., None] * z + b[:, 2][..., None]
    den = a[:, 0][..., None] * z**2 + a[:, 1][..., None] * z + a[:, 2][..., None]

    return num / (den + 1e-10)

In [ ]:
# | export


def compute_spectrogram(
    signal: np.ndarray,
    sample_rate: int,
    n_fft: int,
    hop_length: int,
    max_freq: float = None,
    time_limit_s: float = 0.5,
):
    # librosa will be removed in the future
    import librosa

    if max_freq is None:
        max_freq = sample_rate // 2

    S = librosa.stft(
        signal,
        n_fft=n_fft,
        hop_length=hop_length,
        center=False,
        window="hann",
    )
    S_db = librosa.amplitude_to_db(np.abs(S), ref=np.max)

    time_limit = int((time_limit_s * sample_rate) / hop_length)
    freq_limit = int(max_freq / (sample_rate / 2) * (n_fft // 2 + 1))

    return S_db[:freq_limit, :time_limit]

In [ ]:
# | export


def display_audio_with_title(audio_data, sample_rate, title):
    """
    Display audio with a title next to it in a Jupyter notebook.
    """
    # Create Audio widget
    audio_widget = Audio(audio_data, rate=sample_rate)

    # Create HTML with flexbox layout
    html = f"""
    <div style="display: flex; align-items: center; margin-bottom: 10px;">
        <div style="flex: 0 0 200px; font-weight: bold;">{title}</div>
        <div>{audio_widget._repr_html_()}</div>
    </div>
    """

    display(HTML(html))